KURULUMLAR

In [1]:
!pip install accelerate==0.25.0 torchmetrics==1.2.1 tqdm==4.66.1 transformers==4.36.2 diffusers==0.25.0 einops==0.7.0 bitsandbytes==0.39.0 scipy==1.11.1 opencv-python gradio==4.24.0 fvcore cloudpickle omegaconf pycocotools basicsr av onnxruntime==1.16.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.7/872.7 kB 12.8 MB/s eta 0:00:00


In [2]:
!pip install firebase_admin

YOLO ENTEGRESİ

In [4]:
import time
import cv2
import os


from ultralytics import YOLO
import firebase_admin
from firebase_admin import credentials, firestore, storage



#veri yolu için zamana göre isimlendirme
veri_yol = ("/"+str(time.strftime('%c'))).replace(" ","-")


#veri yolu ve görüntü kaydetme sırası için yardımcı bir class, sadece veri tutacak
class Veri:
    yol = ""
    counter = 0

#Bir klasör yok ise oluşturma işlemi
if (not os.path.exists(os.getcwd() + veri_yol)):
    os.mkdir(os.getcwd() + veri_yol)

#yolo modeli yükle
model = YOLO("yolov8n.pt")


#firebase yapılandır
credentialData = credentials.Certificate("hackathonproject-5e6ef-firebase-adminsdk-mubj0-d18674376c.json")
firebase_admin.initialize_app(credentialData, {
'storageBucket': 'hackathonproject-5e6ef.appspot.com'
})





ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [9]:

#bilgileri buluta yükle
def firebase_veri_yukle(gorsel_Adi,nesne_sayi,tumBilgi):

    firestoreDb = firestore.client()
    bucket = storage.bucket()


    #document sayısını öğrenme
    veri_document_sayi = firestoreDb.collection("urunVeri").get()

    #firebase tarih bilgisi
    timestamp = firestore.SERVER_TIMESTAMP

    #veri girişi
    deger = len(veri_document_sayi)+1
    belge_ref = firestoreDb.collection("urunVeri").document("urunBilgi"+str(deger))


    #veriler
    urun_verileri = {
            'ID':str(deger),
            'gorsel_adi': "urun_Gorseller"+f"{gorsel_Adi}",
            'konum':"Yozgat/Boğazlıyan",
            'urun_kategori':nesne_sayi,
            'zaman': timestamp,
            "tumBilgi":tumBilgi,#YOLO bilgileri
        }

    #bilgi_upload
    belge_ref.set(urun_verileri, merge=True)

    #gorsel yükleme
    blob = bucket.blob("urun_gorseller"+gorsel_Adi)
    blob.upload_from_filename(gorsel_Adi)






#tahmin için gerekli fonskiyon
def goruntu_Tahmin(goruntu_yol):
    results = model([goruntu_yol])
    # Process results list
    for result in results:
        boxes = result.boxes
        print("\n\n\nBAŞLADI")
        print("HEPSİ BOXES")
        print(str(result.boxes))
        print("TUM BİLGİ")
        print(str(result))
        print("TİP")
        print(type(result.boxes))
        print("CLASS BİLGİSİ")
        print(result.boxes.cls)
        print("Toplam nesne sayısı : "+str(len(result.boxes.cls)))
        print("CLASS ID BİLGİSİ")
        print(result.boxes.id)
        print("BİTTİ\n\n\n")
        result.save(filename=goruntu_yol)

        firebase_veri_yukle(goruntu_yol,str(len(result.boxes.cls)),str(result.boxes))


def YOLO_islem(gorsel_yolu):
    #tahmin yapma
    goruntu_Tahmin(os.getcwd()+ "/" + gorsel_yolu)

    #yolları yazdır
    print("Veri yol")
    print(gorsel_yolu)


MODEL ÇALIŞTIRMA

In [10]:
YOLO_islem("gorsel.jpg")


0: 640x480 1 bench, 299.4ms
Speed: 8.2ms preprocess, 299.4ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 480)



BAŞLADI
HEPSİ BOXES
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([13.])
conf: tensor([0.2882])
data: tensor([[6.1020e+01, 2.3793e+02, 5.0474e+02, 6.2169e+02, 2.8817e-01, 1.3000e+01]])
id: None
is_track: False
orig_shape: (788, 557)
shape: torch.Size([1, 6])
xywh: tensor([[282.8787, 429.8102, 443.7175, 383.7671]])
xywhn: tensor([[0.5079, 0.5454, 0.7966, 0.4870]])
xyxy: tensor([[ 61.0200, 237.9267, 504.7374, 621.6937]])
xyxyn: tensor([[0.1096, 0.3019, 0.9062, 0.7890]])
TUM BİLGİ
ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: